# Assignment 1 - Building a Sudoku Solver in Python

This assignment is part of the [Zero to Data Science Bootcamp by Jovian](https://jovian.ai/learn/zero-to-data-analyst-bootcamp).


## Introduction

In this assignment, we'll write a Python program which can solve a Sudoku, a popular Japanese puzzle you may have seen in newspapers. Here's what a Sudoku looks like:

<img src="https://i.imgur.com/vfArdnW.jpg" width="360">

It's a 9x9 grid containing several blank spaces and some numbers (between 1 and 9). There are also nine 3x3 subgrids (indicated by the dark lines).


> **Solving a Sudoku**: To _solve_ a Sudoku, you must fill all the blank spaces in the above 9x9 grid with digits so that each column, each row and each of the nine 3x3 subgrids (also called "boxes") contain all of the digits from 1 to 9, without repetition. 

Here's the solution to the above puzzle:

<img src="https://i.imgur.com/0oXXRNk.png" width="360">

Can you verify that this solution matches the criteria mentioned above?

> **Sudoku World Record**: In 2018, China's Wang Shiyao set a new world record in Sudoku on Wednesday when she managed to solve a 9x9 sudoku grid in 54.44 seconds at the World Sudoku & Puzzle Championship held in Prague. 

Let's see if we can beat that record with a Python program, without solving even a single Sudoku by hand.

Here are the steps we'll follow to create a Sudoku solver in Python:

1. Represent a Sudoku as a list of lists in Python
2. Create helper functions to extract rows, columns and boxes from the Sudoku
3. Create functions to check if a Sudoku is valid or complete
4. Use a recursive strategy to solve a Sudoku by trial & error
5. Read 100 Sudokus from a file and solve them all together

## 1. Puzzle Representation 

The first step for solving any real-world problem is to figure out the representation for the inputs and outputs of the problem. In this case, the input is an unsolved Sudoku puzzle and the output is the solved version of the input puzzles

We'll use a list of lists of numbers to represent a Sudoku puzzle. 

<img src="https://i.imgur.com/vfArdnW.jpg" width="360">

Here's how we can represent the above puzzle:

In [4]:
puzzle1 = [[5, 3, 0, 0, 7, 0, 0, 0, 0], 
           [6, 0, 0, 1, 9, 5, 0, 0, 0], 
           [0, 9, 8, 0, 0, 0, 0, 6, 0], 
           [8, 0, 0, 0, 6, 0, 0, 0, 3],
           [4, 0, 0, 8, 0, 3, 0, 0, 1],
           [7, 0, 0, 0, 2, 0, 0, 0, 6],
           [0, 6, 0, 0, 0, 0, 2, 8, 0],
           [0, 0, 0, 4, 1, 9, 0, 0, 5],
           [0, 0, 0, 0, 8, 0, 0, 7, 9]]

In [5]:
puzzle1

[[5, 3, 0, 0, 7, 0, 0, 0, 0],
 [6, 0, 0, 1, 9, 5, 0, 0, 0],
 [0, 9, 8, 0, 0, 0, 0, 6, 0],
 [8, 0, 0, 0, 6, 0, 0, 0, 3],
 [4, 0, 0, 8, 0, 3, 0, 0, 1],
 [7, 0, 0, 0, 2, 0, 0, 0, 6],
 [0, 6, 0, 0, 0, 0, 2, 8, 0],
 [0, 0, 0, 4, 1, 9, 0, 0, 5],
 [0, 0, 0, 0, 8, 0, 0, 7, 9]]



Note the following details about the above representation:

- The outer list contains 9 elements, one for each row of the puzzle
- Each element in the outer list is itself a list, containing 9 elements, one for each column
- Blank spaces in the Sudoku are represented using `0` and filled spaces are represented using digits.

We can check the number of rows and columns using the `len` function.

In [6]:
# Number of rows
len(puzzle1)

9

In [7]:
# Number of elements row no. 0
len(puzzle1[0])

9

We can access a row or a single value using the list indexing notation. Recall that list elements have indices from `0` to `n-1`, for a list of length `n`.

In [8]:
# Row no. 0
puzzle1[0]

[5, 3, 0, 0, 7, 0, 0, 0, 0]

In [9]:
# Element at Row no. 2 and col no. 2 (counting from 0)
puzzle1[2][2]

8

Like the unsolved puzzle, the solved Sudoku can also be represented as a list of lists. Here's the solution to the above puzzle:

<img src="https://i.imgur.com/0oXXRNk.png" width="360">

> **QUESTION 1**: Represent the above solved Sudoku using a list of lists, in a similar fashion as the unsolved Sudoku.


In [10]:
solution1 = [[5, 3, 4, 6, 7, 8, 9, 1, 2], 
           [6, 7, 2, 1, 9, 5, 3, 4, 8], 
           [1, 9, 8, 3, 4, 2, 5, 6, 7], 
           [8, 5, 9, 7, 6, 1, 4, 2, 3],
           [4, 2, 6, 8, 5, 3, 7, 9, 1],
           [7, 1, 3, 9, 2, 4, 8, 5, 6],
           [9, 6, 1, 5, 3, 7, 2, 8, 4],
           [2, 8, 7, 4, 1, 9, 6, 3, 5],
           [3, 4, 5, 2, 8, 6, 1, 7, 9]]

In [11]:
solution1

[[5, 3, 4, 6, 7, 8, 9, 1, 2],
 [6, 7, 2, 1, 9, 5, 3, 4, 8],
 [1, 9, 8, 3, 4, 2, 5, 6, 7],
 [8, 5, 9, 7, 6, 1, 4, 2, 3],
 [4, 2, 6, 8, 5, 3, 7, 9, 1],
 [7, 1, 3, 9, 2, 4, 8, 5, 6],
 [9, 6, 1, 5, 3, 7, 2, 8, 4],
 [2, 8, 7, 4, 1, 9, 6, 3, 5],
 [3, 4, 5, 2, 8, 6, 1, 7, 9]]

The following cell should output `True` if your definition of `solution1` is correct.

In [12]:
len(solution1) == 9 and len(solution1[0]) == 9

True



> **QUESTION 2**: Retrieve row no. 3 of the solution (counting from 0) using the list indexing notation.

In [13]:
row3 = solution1[3]

In [14]:
row3

[8, 5, 9, 7, 6, 1, 4, 2, 3]

The following cell should output `True` if your definition of `solution1` is correct.

In [15]:
row3 == [8, 5, 9, 7, 6, 1, 4, 2, 3]

True



> **QUESTION 3**: Retrieve the value in row no. 4 and column no. 5 of the solution (both counting from 0).

In [16]:
val_4_5 = solution1[4][5]

In [17]:
val_4_5

3

The following cell should output `True` if your definition of `solution1` is correct.

In [18]:
val_4_5 == 3

True

> **QUESTION 4**: Retrieve the value in the last row and column no. 0 of the solution (counting from 0).

In [19]:
val_last_zero = solution1[-1][0]

In [20]:
val_last_zero

3

The following cell should output `True` if your definition of `solution1` is correct.

In [21]:
val_last_zero == 3

True

## 2. Extracting Rows, Columns and Boxes

Before we can solve a Sudoku, we'll need a way to extract specific rows, columns and boxes from the Sudoku. We'll create a helper function for each of these. 

### Rows

<img src="https://i.imgur.com/FR98oSb.jpg" width="360">

> **QUESTION 5**: Write a function to extract row no. k (counting from 0) of a Sudoku as a list of numbers. Rows are numbered 0 to 8, starting from the top. E.g. row no. 2 above is `[0, 9, 8, 0, 0, 0, 0, 6, 0]`.
     
     


In [22]:
def get_row(sudoku, k):
    return sudoku[k]

In [23]:
get_row(puzzle1, 2)

[0, 9, 8, 0, 0, 0, 0, 6, 0]

In [24]:
get_row(solution1, 3)

[8, 5, 9, 7, 6, 1, 4, 2, 3]

The following cell should output `True` if your definition is correct.

In [25]:
get_row(solution1, 3) == [8, 5, 9, 7, 6, 1, 4, 2, 3]

True

In [26]:
get_row(solution1, -1)

[3, 4, 5, 2, 8, 6, 1, 7, 9]


### Columns

<img src="https://i.imgur.com/FezcTVP.png" width="360">

> **QUESTION 6**: Write a function to extract column no. k of a Sudoku as a list of numbers. Columns are numbered 0 to 8 starting from the left. E.g. column no. 4 above is `[7, 9, 0, 6, 0, 2, 0, 1, 8]`.
>
> *Hint*: Use a `for` loop or list comprehension to get the k-th element of each row.



In [27]:
def get_col(sudoku, k):
    #col = []
    #for row in sudoku:
    #    col.append(row[k])
    #return col
    return [row[k] for row in sudoku]

In [28]:
get_col(puzzle1, 4)

[7, 9, 0, 6, 0, 2, 0, 1, 8]

In [29]:
get_col(solution1, 5)

[8, 5, 2, 1, 3, 4, 7, 9, 6]

In [30]:
solution1

[[5, 3, 4, 6, 7, 8, 9, 1, 2],
 [6, 7, 2, 1, 9, 5, 3, 4, 8],
 [1, 9, 8, 3, 4, 2, 5, 6, 7],
 [8, 5, 9, 7, 6, 1, 4, 2, 3],
 [4, 2, 6, 8, 5, 3, 7, 9, 1],
 [7, 1, 3, 9, 2, 4, 8, 5, 6],
 [9, 6, 1, 5, 3, 7, 2, 8, 4],
 [2, 8, 7, 4, 1, 9, 6, 3, 5],
 [3, 4, 5, 2, 8, 6, 1, 7, 9]]

The following cell should output `True` if your implementation is correct.

In [31]:
get_col(solution1, 5) == [8, 5, 2, 1, 3, 4, 7, 9, 6]

True

You can use the cells below to test your implementation with a few more cases.

In [32]:
get_col(solution1, 2)

[4, 2, 8, 9, 6, 3, 1, 7, 5]

In [33]:
get_col(solution1,-1)

[2, 8, 7, 3, 1, 6, 4, 5, 9]

### Boxes

<img src="https://i.imgur.com/n8wkXEo.jpg" width="360">

> **QUESTION 7**: Each 3x3 subgrid of the Sudoku is called a box. Write a function to extract the box no. k of a Sudoku as a list of numbers. Boxes are numbered from 0 to 8 as shown above. The numbers in a box are represented as a list, going from left to right and top to bottom. E.g. box no. 0 above is `[5, 3, 0, 6, 0, 0, 0, 9, 8]`.
> 
> *Hint*: Use `if-elif-else` statements to select the starting index of the box and list concatenation to join the 3 rows in a box into a single row.

In [34]:
def get_box(sudoku, k):
    i,j = (k//3)*3 ,(k%3)*3
    return sudoku[i][j:j+3] + sudoku[i+1][j:j+3] + sudoku[i+2][j:j+3]

In [35]:
get_box(puzzle1, 0)

[5, 3, 0, 6, 0, 0, 0, 9, 8]

In [36]:
get_box(solution1, 7)

[5, 3, 7, 4, 1, 9, 2, 8, 6]

The following cell should output `True` if your implementation is correct.

In [37]:
get_box(solution1, 7) == [5, 3, 7, 4, 1, 9, 2, 8, 6]

True

You can use the cells below to test your implementation with a few more cases.

In [38]:
get_box(solution1,-1)

[2, 8, 4, 6, 3, 5, 1, 7, 9]

In [39]:
jovian.commit()

NameError: ignored

### First Empty Position

To start filling the Sudoku, we need to find an empty position to fill.

> **QUESTION 8**: Write a function which finds the row & column index of the first empty position (indicated by 0) within a Sudoku. If the row no. i and column no. j column is the first empty position, the function should return the tuple `i, j`. If there are no empty positions, return `None, None`.

In [40]:
def first_empty_position(sudoku):
    for i in range(9):
        for j in range(9):
            if sudoku[i][j] == 0:
                return i,j
    return None,None


In [41]:
first_empty_position(puzzle1)

(0, 2)

In [42]:
first_empty_position(solution1)

(None, None)

If your implementation is correct, the following cell should return `True`.

In [43]:
first_empty_position(puzzle1) == (0, 2)

True

In [44]:
first_empty_position(solution1) == (None, None)

True

## 3. Sudoku Validations 


### Valid Sudoku

A Sudoku puzzle is valid if none of the rows, columns, or boxes contains repeating digits. For example, if a row of a Sudoku contains the number 5 twice, then the Sudoku puzzle is invalid and can't be solved. The same holds true for columns and boxes.


![](https://i.imgur.com/QfvkcsM.png)


First, we'll create a helper function to check if a row/column/box in a Sudoku is valid.

> **QUESTION 9**: Write a function to check if a list of 9 numbers (containing digits from 1 to 9 and 0s to indicate blank spaces) is a valid section (row, column or box) for a Sudoku. Only 0 can occur more than once, the numbers 1 to 9 can occur at most once. Your function should return `True` if the section is valid and `False` otherwise.
>
> *Hint*: You may find the `count` method of a list useful.

In [46]:
def is_section_valid(nums):
    section_dict = {}
    for num in nums:
        if num == 0 : 
            continue
        elif num not in section_dict:
            section_dict[num] = nums.count(num)
    return not (max(section_dict.values()) > 1)
    

In [47]:
# should return True
is_section_valid([5, 3, 7, 4, 1, 9, 2, 8, 6])

True

In [48]:
# should return True
is_section_valid([5, 3, 0, 6, 0, 0, 0, 9, 8])

True

In [49]:
# should return False
is_section_valid([5, 3, 0, 6, 0, 8, 0, 9, 8])

False

We can now use the `is_section_valid` function to check if each row, column and box is valid. Rows, columns and boxes are retrieved using the `get_row`, `get_column` and `get_box` functions defined earlier.

Let's create a function `is_sudoku_valid` to bring it all together.

In [50]:
def is_sudoku_valid(sudoku):
    rows_valid = all([is_section_valid(get_row(sudoku, i)) for i in range(0, 9)])
    cols_valid = all([is_section_valid(get_col(sudoku, i)) for i in range(0, 9)])
    boxes_valid = all([is_section_valid(get_box(sudoku, i)) for i in range(0, 9)])
    return rows_valid and cols_valid and boxes_valid

In [51]:
# Valid Puzzle
puzzle2 = [[5, 3, 0, 0, 7, 0, 0, 0, 0], 
           [6, 0, 0, 1, 9, 5, 0, 0, 0], 
           [0, 9, 8, 0, 4, 0, 0, 6, 0], 
           [8, 0, 0, 0, 6, 0, 0, 0, 3],
           [4, 0, 0, 8, 0, 3, 0, 0, 1],
           [7, 0, 0, 0, 2, 0, 0, 0, 6],
           [0, 6, 0, 0, 0, 0, 2, 8, 0],
           [0, 0, 0, 4, 1, 9, 0, 0, 5],
           [0, 0, 0, 0, 8, 0, 0, 7, 9]]

In [52]:
# Invalid Puzzle
puzzle2 = [[5, 3, 0, 0, 7, 0, 0, 0, 0], 
           [6, 0, 0, 1, 9, 5, 0, 0, 0], 
           [0, 9, 8, 0, 8, 0, 0, 6, 0], 
           [8, 0, 0, 0, 6, 0, 0, 0, 3],
           [4, 0, 0, 8, 0, 3, 0, 0, 1],
           [7, 0, 0, 0, 2, 0, 0, 0, 6],
           [0, 6, 0, 0, 0, 0, 2, 8, 0],
           [0, 0, 0, 4, 1, 9, 0, 0, 5],
           [0, 0, 0, 0, 8, 0, 0, 7, 9]]

Check your implementation by running the cells below.

In [53]:
# should return True
is_sudoku_valid(puzzle1)

True

In [54]:
# should return False
is_sudoku_valid(puzzle2)

False

You can use the cells below to test `is_sudoku_valid` with a few more cases.

In [55]:
is_sudoku_valid(solution1)

True

### Complete/Solved Sudoku

Next, we need a way to check if a Sudoku is completely solved. This can be done by checking that each row, each column and each box in the Sudoku contains all the numbers from 1 to 9 exactly once.

![](https://i.imgur.com/uG2uDk7.png)

> **QUESTION 10**: Write a function to check if a list of 9 numbers (containing digits from 1 to 9) represents a complete section (row, column or box) for a Sudoku. The list should contain all the numbers from 1 to 9 exactly once. Your function should return `True` if the section is complete and `False` otherwise.
>
> *Hint*: You may find the `count` method of a list useful.


In [56]:
def is_section_complete(nums):
    for num in range(1,10):
        if nums.count(num) != 1:
            return False
    return True

In [57]:
# should return False
is_section_complete([0, 9, 8, 0, 0, 0, 0, 6, 0])

False

In [58]:
# should return True
is_section_complete([1, 9, 8, 3, 4, 2, 5, 6, 7])

True

You can use the cells below to test your implementation with a few more cases.

In [59]:
is_section_complete([5, 3, 0, 0, 7, 0, 0, 0, 0])

False

In [60]:
is_section_complete([5, 3, 1, 2, 7, 4, 6, 8, 9])

True

We can now use the `is_section_complete` function to check if each row, column and box is complete. Rows, columns and boxes are retrieved using the `get_row`, `get_column` and `get_box` functions defined earlier.

Let's create a function `is_sudoku_complete` to bring it all together and check if an entire Sudoku is complete/solved.

In [61]:
def is_sudoku_complete(sudoku):
    rows_complete = all([is_section_complete(get_row(sudoku, i)) for i in range(0, 9)])
    cols_complete = all([is_section_complete(get_col(sudoku, i)) for i in range(0, 9)])
    boxes_complete = all([is_section_complete(get_box(sudoku, i)) for i in range(0, 9)])
    return rows_complete and cols_complete and boxes_complete

Check your implementation of `is_section_complete` using the cells below.

In [62]:
# should return False
is_sudoku_complete(puzzle1)

False

In [63]:
# should return True
is_sudoku_complete(solution1)

True

## 4. Recursive Solution

Now we have all the components to start building our Sudoku solver. Our solver will follow the simple approach of trying all possible solutions for filling the blank spaces one by one, while making sure that the Sudoku remains valid. 

We'll use a technique called recursion, which is best understood by working backwards. Consider the following scenarios:

* **No empty spaces**: If a Sudoku has no empty spaces, then we can simply check if the Sudoku is already complete/solved using `is_sudoku_complete`. The Suduko is either already solved or invalid.

* **1 empty space**: If a Sudoku has just one empty space, we can try to insert each digit from 1 to 9 into the empty space, and verify which digit, if any, leads to a completed solved Sudoku.

* **2 empty spaces**: If a Sudoku has two empty spaces, we can try to insert each digit from 1 to 9 one-by-one into the first empty space, while making sure the Sudoku remains valid. For each valid attempt at inserting a number, the puzzle reduces to the previous problem of solving a Sudoku with just two empty spaces.

* **3 empty spaces**: If a Sudoku has three empty spaces, we can try to insert each digit from 1 to 9 one-by-one into the first empty space, while making sure the Sudoku remains valid. For each valid attempt at inserting a number, the puzzle reduces to the previous problem of solving a Sudoku with two empty spaces.

* and so on....

* **n empty spaces**: If a Sudokuk has `n` empty spaces, we can try to insert each digit from 1 to 9 one-by-one into the first empty space, while making sure the Sudoku remains valid. For each valid attempt at inserting a number, the puzzle reduces to the previous problem of solving a Sudoku with `n-1` empty spaces.

Here's a quick tutorial on recursion: https://youtu.be/wMNrSM5RFMc

Let's define a helper function `repeat` to implement the above strategy for any number of empty spaces. The function repeat will attempt to fill the first empty space within a Sudoku and invoke itself to fill the remaining spaces *recursively* i.e. by invoking itself with a different input. The function will return `True` if it was able to fill all the spaces successfully, otherwise it will return `False`.


In [64]:
def repeat(sudoku):
    # Check if Sudoku is already complete
    if is_sudoku_complete(sudoku):
        return True
    
    # Find the first empty position
    i, j = first_empty_position(sudoku)
    
    # Try to fill it with numbers 1 to 9
    for digit in range(1, 10):
        
        # Insert the digit into the right place
        sudoku[i][j] = digit
        
        # Check if the new puzzle is valid
        if is_sudoku_valid(sudoku):
            
            # Try to fill the remaining spaces recursively using `repeat`
            # Node that this will directly fill values into the sudoku
            result = repeat(sudoku)
            
            # If the recursive result is true, we have found the answer and filled the sudoku
            if result is True:
                return True
        
        
        # Remove the digit, it doesn't lead to a solution
        sudoku[i][j] = 0
        
    
    # There are no valid numbers to fill the empty slot(s)
    return False

Note that `repeat` directly makes changes inside the puzzle passed to it, so it does not need to return the puzzle itself. Do you see how repeat works? Here's a visualization of the process for a 4x4 sudoku (4 rows, columns & boxes instead of 9): 

![](https://i.imgur.com/Njy5BtB.jpg)


Finally, we can create a `solve_sudoku` function which uses `repeat` to solve a Sudoku and returns the solved version (or `None` if the Sudoku is unsolvable).

In [65]:
import copy

def solve_sudoku(sudoku):
    # Create a deep copy of the puzzle (list of lists),
    # to avoid modifying the original
    copied_sudoku = copy.deepcopy(sudoku)
    
    # Try to complete the Sudoku using repeat
    result = repeat(copied_sudoku)
    
    # Return the solved version if successful
    if result is True:
        return copied_sudoku
    
    # Return None if unsuccessful
    return None

Let's test it out!

In [66]:
puzzle1

[[5, 3, 0, 0, 7, 0, 0, 0, 0],
 [6, 0, 0, 1, 9, 5, 0, 0, 0],
 [0, 9, 8, 0, 0, 0, 0, 6, 0],
 [8, 0, 0, 0, 6, 0, 0, 0, 3],
 [4, 0, 0, 8, 0, 3, 0, 0, 1],
 [7, 0, 0, 0, 2, 0, 0, 0, 6],
 [0, 6, 0, 0, 0, 0, 2, 8, 0],
 [0, 0, 0, 4, 1, 9, 0, 0, 5],
 [0, 0, 0, 0, 8, 0, 0, 7, 9]]

In [67]:
%%time
puzzle1_solved = solve_sudoku(puzzle1)
puzzle1_solved

CPU times: user 3.91 s, sys: 16.4 ms, total: 3.93 s
Wall time: 3.98 s


[[5, 3, 4, 6, 7, 8, 9, 1, 2],
 [6, 7, 2, 1, 9, 5, 3, 4, 8],
 [1, 9, 8, 3, 4, 2, 5, 6, 7],
 [8, 5, 9, 7, 6, 1, 4, 2, 3],
 [4, 2, 6, 8, 5, 3, 7, 9, 1],
 [7, 1, 3, 9, 2, 4, 8, 5, 6],
 [9, 6, 1, 5, 3, 7, 2, 8, 4],
 [2, 8, 7, 4, 1, 9, 6, 3, 5],
 [3, 4, 5, 2, 8, 6, 1, 7, 9]]

In [68]:
solution1

[[5, 3, 4, 6, 7, 8, 9, 1, 2],
 [6, 7, 2, 1, 9, 5, 3, 4, 8],
 [1, 9, 8, 3, 4, 2, 5, 6, 7],
 [8, 5, 9, 7, 6, 1, 4, 2, 3],
 [4, 2, 6, 8, 5, 3, 7, 9, 1],
 [7, 1, 3, 9, 2, 4, 8, 5, 6],
 [9, 6, 1, 5, 3, 7, 2, 8, 4],
 [2, 8, 7, 4, 1, 9, 6, 3, 5],
 [3, 4, 5, 2, 8, 6, 1, 7, 9]]

In [69]:
puzzle1_solved == solution1

True

Do you see how the `repeat` function works, by repeatedly invoking itself with progressively easier problems? Recursion can be tricky to wrap your head around at first, but it's a very powerful concept once you understand it.


## (Optional) Factorial of Numbers using Recursion


Here's a simpler example of recursion: finding the factorial of a number. Factorial of a number `n` is defined as the product of all numbers from `1` to `n`.

> **(Optional) QUESTION 11:** Write a recursive function to compute the factorial of a number `n`. The factorial of 0 is 1 and the factorial of any number `n` greater than zero is the product of `n * factorial(n-1)`.

In [70]:
def factorial(n):
    if n == 0 or n == 1:
        return 1
    return n * factorial(n-1)

In [71]:
factorial(10)

3628800

The factorial of 10 is computed using the factorial of 9, which itself is computed using the factorial of 8 and so on. 

## (Optional) Solving hundreds of Sudokus 

Our `solve_sudoku` function is generic enough that it can solve any Sudoku. In this optional extension to the assignment, we'll download a file containing 100 Sudoku puzzles, process the file to create Sudokus is our list-of-lists representation, solve all the puzzles and finally write the results back to a file.

First, let's download the file:

In [72]:
sudokus_url = 'https://gist.githubusercontent.com/aakashns/033af5f9f6f2ec3a2f322105dad38c01/raw/7af74a86ee7fd9ec9bbb9d3b5a2bf08e9e080532/hundred_sudokus.csv'

In [73]:
from urllib.request import urlretrieve

In [74]:
urlretrieve(sudokus_url, 'sudokus.csv')

('sudokus.csv', <http.client.HTTPMessage at 0x7fab2b976290>)

In [75]:
with open('sudokus.csv', 'r') as f:
    lines = [l.strip() for l in f.readlines()]

In [76]:
len(lines)

100

In [77]:
lines[:5]

['004300209005009001070060043006002087190007400050083000600000105003508690042910300',
 '040100050107003960520008000000000017000906800803050620090060543600080700250097100',
 '600120384008459072000006005000264030070080006940003000310000050089700000502000190',
 '497200000100400005000016098620300040300900000001072600002005870000600004530097061',
 '005910308009403060027500100030000201000820007006007004000080000640150700890000420']

Each line of the file represents a Sudoku. Let's create a helper function to convert a line from the file into a list of lists, the representation we have been using so far.


> **(Optional) QUESTION 12**: Write a function `parse_sudoku` to convert a Sudoku into a list of lists

In [78]:
def parse_sudoku(sudoku_str):
    sudoko = []
    for i in range(9):
        line,j = [],i*9
        for j in range(j,j+9):
            line.append(int(sudoku_str[j]))
        sudoko.append(line)
    return sudoko
        

The following cell should output `True` if your implementation is correct.

In [79]:
sudoku_str1 = '004300209005009001070060043006002087190007400050083000600000105003508690042910300';

sudoku_parsed1 = [[0, 0, 4, 3, 0, 0, 2, 0, 9],
                  [0, 0, 5, 0, 0, 9, 0, 0, 1],
                  [0, 7, 0, 0, 6, 0, 0, 4, 3],
                  [0, 0, 6, 0, 0, 2, 0, 8, 7],
                  [1, 9, 0, 0, 0, 7, 4, 0, 0],
                  [0, 5, 0, 0, 8, 3, 0, 0, 0],
                  [6, 0, 0, 0, 0, 0, 1, 0, 5],
                  [0, 0, 3, 5, 0, 8, 6, 9, 0],
                  [0, 4, 2, 9, 1, 0, 3, 0, 0]]

parse_sudoku(sudoku_str1) == sudoku_parsed1

True


We can now use list comprehension to convert into a list of lists.


In [80]:
sudokus = [parse_sudoku(line) for line in lines]    

In [81]:
sudokus[:5]

[[[0, 0, 4, 3, 0, 0, 2, 0, 9],
  [0, 0, 5, 0, 0, 9, 0, 0, 1],
  [0, 7, 0, 0, 6, 0, 0, 4, 3],
  [0, 0, 6, 0, 0, 2, 0, 8, 7],
  [1, 9, 0, 0, 0, 7, 4, 0, 0],
  [0, 5, 0, 0, 8, 3, 0, 0, 0],
  [6, 0, 0, 0, 0, 0, 1, 0, 5],
  [0, 0, 3, 5, 0, 8, 6, 9, 0],
  [0, 4, 2, 9, 1, 0, 3, 0, 0]],
 [[0, 4, 0, 1, 0, 0, 0, 5, 0],
  [1, 0, 7, 0, 0, 3, 9, 6, 0],
  [5, 2, 0, 0, 0, 8, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 1, 7],
  [0, 0, 0, 9, 0, 6, 8, 0, 0],
  [8, 0, 3, 0, 5, 0, 6, 2, 0],
  [0, 9, 0, 0, 6, 0, 5, 4, 3],
  [6, 0, 0, 0, 8, 0, 7, 0, 0],
  [2, 5, 0, 0, 9, 7, 1, 0, 0]],
 [[6, 0, 0, 1, 2, 0, 3, 8, 4],
  [0, 0, 8, 4, 5, 9, 0, 7, 2],
  [0, 0, 0, 0, 0, 6, 0, 0, 5],
  [0, 0, 0, 2, 6, 4, 0, 3, 0],
  [0, 7, 0, 0, 8, 0, 0, 0, 6],
  [9, 4, 0, 0, 0, 3, 0, 0, 0],
  [3, 1, 0, 0, 0, 0, 0, 5, 0],
  [0, 8, 9, 7, 0, 0, 0, 0, 0],
  [5, 0, 2, 0, 0, 0, 1, 9, 0]],
 [[4, 9, 7, 2, 0, 0, 0, 0, 0],
  [1, 0, 0, 4, 0, 0, 0, 0, 5],
  [0, 0, 0, 0, 1, 6, 0, 9, 8],
  [6, 2, 0, 3, 0, 0, 0, 4, 0],
  [3, 0, 0, 9, 0, 0, 0, 0, 0],
  [0,

We can also use list comprehension to solve all the puzzles.

In [82]:
solved_sudokus = [solve_sudoku(sudoku) for sudoku in sudokus]

In [83]:
solved_sudokus[:5]

[[[8, 6, 4, 3, 7, 1, 2, 5, 9],
  [3, 2, 5, 8, 4, 9, 7, 6, 1],
  [9, 7, 1, 2, 6, 5, 8, 4, 3],
  [4, 3, 6, 1, 9, 2, 5, 8, 7],
  [1, 9, 8, 6, 5, 7, 4, 3, 2],
  [2, 5, 7, 4, 8, 3, 9, 1, 6],
  [6, 8, 9, 7, 3, 4, 1, 2, 5],
  [7, 1, 3, 5, 2, 8, 6, 9, 4],
  [5, 4, 2, 9, 1, 6, 3, 7, 8]],
 [[3, 4, 6, 1, 7, 9, 2, 5, 8],
  [1, 8, 7, 5, 2, 3, 9, 6, 4],
  [5, 2, 9, 6, 4, 8, 3, 7, 1],
  [9, 6, 5, 8, 3, 2, 4, 1, 7],
  [4, 7, 2, 9, 1, 6, 8, 3, 5],
  [8, 1, 3, 7, 5, 4, 6, 2, 9],
  [7, 9, 8, 2, 6, 1, 5, 4, 3],
  [6, 3, 1, 4, 8, 5, 7, 9, 2],
  [2, 5, 4, 3, 9, 7, 1, 8, 6]],
 [[6, 9, 5, 1, 2, 7, 3, 8, 4],
  [1, 3, 8, 4, 5, 9, 6, 7, 2],
  [7, 2, 4, 8, 3, 6, 9, 1, 5],
  [8, 5, 1, 2, 6, 4, 7, 3, 9],
  [2, 7, 3, 9, 8, 1, 5, 4, 6],
  [9, 4, 6, 5, 7, 3, 8, 2, 1],
  [3, 1, 7, 6, 9, 2, 4, 5, 8],
  [4, 8, 9, 7, 1, 5, 2, 6, 3],
  [5, 6, 2, 3, 4, 8, 1, 9, 7]],
 [[4, 9, 7, 2, 5, 8, 3, 1, 6],
  [1, 8, 6, 4, 3, 9, 7, 2, 5],
  [2, 5, 3, 7, 1, 6, 4, 9, 8],
  [6, 2, 9, 3, 8, 1, 5, 4, 7],
  [3, 7, 5, 9, 6, 4, 1, 8, 2],
  [8,

> **(Optional) QUESTION 13**: Write a function `write_results` which writes the solved sudokus to a file.

In [84]:
def write_results(solved_sudokus, filename):
    with open(filename,"w") as f:
        f.write(str(solved_sudokus))

In [85]:
write_results(solved_sudokus, 'sudokus_solved.csv')

Let's view the file to ensure that it was written properly.

In [86]:
with open('sudokus_solved.csv', 'r') as f:
    lines2 = [l.strip() for l in f.readlines()]
    
lines2[:5]

['[[[8, 6, 4, 3, 7, 1, 2, 5, 9], [3, 2, 5, 8, 4, 9, 7, 6, 1], [9, 7, 1, 2, 6, 5, 8, 4, 3], [4, 3, 6, 1, 9, 2, 5, 8, 7], [1, 9, 8, 6, 5, 7, 4, 3, 2], [2, 5, 7, 4, 8, 3, 9, 1, 6], [6, 8, 9, 7, 3, 4, 1, 2, 5], [7, 1, 3, 5, 2, 8, 6, 9, 4], [5, 4, 2, 9, 1, 6, 3, 7, 8]], [[3, 4, 6, 1, 7, 9, 2, 5, 8], [1, 8, 7, 5, 2, 3, 9, 6, 4], [5, 2, 9, 6, 4, 8, 3, 7, 1], [9, 6, 5, 8, 3, 2, 4, 1, 7], [4, 7, 2, 9, 1, 6, 8, 3, 5], [8, 1, 3, 7, 5, 4, 6, 2, 9], [7, 9, 8, 2, 6, 1, 5, 4, 3], [6, 3, 1, 4, 8, 5, 7, 9, 2], [2, 5, 4, 3, 9, 7, 1, 8, 6]], [[6, 9, 5, 1, 2, 7, 3, 8, 4], [1, 3, 8, 4, 5, 9, 6, 7, 2], [7, 2, 4, 8, 3, 6, 9, 1, 5], [8, 5, 1, 2, 6, 4, 7, 3, 9], [2, 7, 3, 9, 8, 1, 5, 4, 6], [9, 4, 6, 5, 7, 3, 8, 2, 1], [3, 1, 7, 6, 9, 2, 4, 5, 8], [4, 8, 9, 7, 1, 5, 2, 6, 3], [5, 6, 2, 3, 4, 8, 1, 9, 7]], [[4, 9, 7, 2, 5, 8, 3, 1, 6], [1, 8, 6, 4, 3, 9, 7, 2, 5], [2, 5, 3, 7, 1, 6, 4, 9, 8], [6, 2, 9, 3, 8, 1, 5, 4, 7], [3, 7, 5, 9, 6, 4, 1, 8, 2], [8, 4, 1, 5, 7, 2, 6, 3, 9], [9, 6, 2, 1, 4, 5, 8, 7, 3], [7, 